## 1. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import difflib

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

## 2. Import Data

In [2]:
#Import data 
anime_data_read = pd.read_csv("C:/Users/PC/Downloads/Dataset/anime.csv")
anime_synopsis = pd.read_csv("C:/Users/PC/Downloads/Dataset/anime_with_synopsis.csv")
anime_data = pd.merge(anime_synopsis , anime_data_read, on='MAL_ID')

#Keep the ori data for second version.
ori_data = anime_data_read

#Rename columns for MAL_ID -> index

anime_data = anime_data.rename(
    columns = {"MAL_ID" : "index" ,"Name_x" : "title", "English name" : "English" , "Japanese name" : "Japanese" }) 
print(anime_data.head())

   index                            title Score_x  \
0      1                     Cowboy Bebop    8.78   
1      5  Cowboy Bebop: Tengoku no Tobira    8.39   
2      6                           Trigun    8.24   
3      7               Witch Hunter Robin    7.27   
4      8                   Bouken Ou Beet    6.98   

                                            Genres_x  \
0    Action, Adventure, Comedy, Drama, Sci-Fi, Space   
1              Action, Drama, Mystery, Sci-Fi, Space   
2  Action, Sci-Fi, Adventure, Comedy, Drama, Shounen   
3  Action, Mystery, Police, Supernatural, Drama, ...   
4          Adventure, Fantasy, Shounen, Supernatural   

                                           sypnopsis  \
0  In the year 2071, humanity has colonized sever...   
1  other day, another bounty—such is the life of ...   
2  Vash the Stampede is the man with a $$60,000,0...   
3  ches are individuals with special powers like ...   
4  It is the dark century and the people are suff...   

       

## 3. Get the Initial Ideas of Data

In [3]:
# list all colums

list(anime_data)

['index',
 'title',
 'Score_x',
 'Genres_x',
 'sypnopsis',
 'Name_y',
 'Score_y',
 'Genres_y',
 'English',
 'Japanese',
 'Type',
 'Episodes',
 'Aired',
 'Premiered',
 'Producers',
 'Licensors',
 'Studios',
 'Source',
 'Duration',
 'Rating',
 'Ranked',
 'Popularity',
 'Members',
 'Favorites',
 'Watching',
 'Completed',
 'On-Hold',
 'Dropped',
 'Plan to Watch',
 'Score-10',
 'Score-9',
 'Score-8',
 'Score-7',
 'Score-6',
 'Score-5',
 'Score-4',
 'Score-3',
 'Score-2',
 'Score-1']

In [4]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16214 entries, 0 to 16213
Data columns (total 39 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          16214 non-null  int64 
 1   title          16214 non-null  object
 2   Score_x        16214 non-null  object
 3   Genres_x       16214 non-null  object
 4   sypnopsis      16206 non-null  object
 5   Name_y         16214 non-null  object
 6   Score_y        16214 non-null  object
 7   Genres_y       16214 non-null  object
 8   English        16214 non-null  object
 9   Japanese       16214 non-null  object
 10  Type           16214 non-null  object
 11  Episodes       16214 non-null  object
 12  Aired          16214 non-null  object
 13  Premiered      16214 non-null  object
 14  Producers      16214 non-null  object
 15  Licensors      16214 non-null  object
 16  Studios        16214 non-null  object
 17  Source         16214 non-null  object
 18  Duration       16214 non-n

In [5]:
#Pre-processing - Selecting attributes

anime_attr = anime_data[["index","title", "Genres_x","English", "Japanese", "Type", "Producers","Licensors","Studios","sypnopsis"]]

#Display 10 rows of df
anime_attr.head(11)

,index,title,Genres_x,English,Japanese,Type,Producers,Licensors,Studios,sypnopsis
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"other day, another bounty—such is the life of ..."
2,6,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,ches are individuals with special powers like ...
4,8,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,"TV Tokyo, Dentsu",Unknown,Toei Animation,It is the dark century and the people are suff...
5,15,Eyeshield 21,"Action, Sports, Comedy, Shounen",Unknown,アイシールド21,TV,"TV Tokyo, Nihon Ad Systems, TV Tokyo Music, Sh...","VIZ Media, Sentai Filmworks",Gallop,Sena is like any other shy kid starting high s...
6,16,Hachimitsu to Clover,"Comedy, Drama, Josei, Romance, Slice of Life",Honey and Clover,ハチミツとクローバー,TV,"Genco, Fuji TV, Shueisha","VIZ Media, Discotek Media",J.C.Staff,"Yuuta Takemoto, a sophomore at an arts college..."
7,17,Hungry Heart: Wild Striker,"Slice of Life, Comedy, Sports, Shounen",Unknown,ハングリーハート Wild Striker,TV,Unknown,Unknown,Nippon Animation,Kyosuke Kano has lived under the shadow of his...
8,18,Initial D Fourth Stage,"Action, Cars, Sports, Drama, Seinen",Unknown,頭文字〈イニシャル〉D FOURTH STAGE,TV,"OB Planning, Studio Jack",Funimation,A.C.G.T.,Takumi Fujiwara finally joins Ryousuke and Kei...
9,19,Monster,"Drama, Horror, Mystery, Police, Psychological,...",Monster,モンスター,TV,"VAP, Shogakukan-Shueisha Productions, Nippon T...",VIZ Media,Madhouse,"Dr. Kenzou Tenma, an elite neurosurgeon recent..."


In [6]:
anime_attr.shape

(16214, 10)

In [7]:
#some df info 

anime_attr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16214 entries, 0 to 16213
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      16214 non-null  int64 
 1   title      16214 non-null  object
 2   Genres_x   16214 non-null  object
 3   English    16214 non-null  object
 4   Japanese   16214 non-null  object
 5   Type       16214 non-null  object
 6   Producers  16214 non-null  object
 7   Licensors  16214 non-null  object
 8   Studios    16214 non-null  object
 9   sypnopsis  16206 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.4+ MB


In [10]:
# Look how many NA values in the data
count_na = anime_attr.isna().sum()
print(count_na)

index        0
title        0
Genres_x     0
English      0
Japanese     0
Type         0
Producers    0
Licensors    0
Studios      0
sypnopsis    0
dtype: int64


In [11]:
# Fill NA with blank string str(' ')
anime_attr["sypnopsis"].fillna(' ', inplace = True)

In [13]:
# Check whether synopsis correctly merged.
anime_attr.head()

,index,title,Genres_x,English,Japanese,Type,Producers,Licensors,Studios,sypnopsis
0,1,Cowboy Bebop,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"other day, another bounty—such is the life of ..."
2,6,Trigun,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,ches are individuals with special powers like ...
4,8,Bouken Ou Beet,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,"TV Tokyo, Dentsu",Unknown,Toei Animation,It is the dark century and the people are suff...


## 4. Selecting Features for Recommendation

In [12]:
#In this case we can see the data are dense so we can move forward to recommendation system by selecting relevant features

selected_features = ["index","title", "Genres_x","English", "Japanese", "Type", "Producers","Licensors","Studios","sypnopsis"]
print(selected_features)

['index', 'title', 'Genres_x', 'English', 'Japanese', 'Type', 'Producers', 'Licensors', 'Studios', 'sypnopsis']


In [14]:
# Combining all features
combined_features = anime_attr['title']+' '+anime_attr['Genres_x']+' '+anime_attr['English']+' '+anime_attr['Japanese']+' '+anime_attr['Type']+' '+anime_attr['Producers']+' '+anime_attr['Licensors']+' '+anime_attr['Studios']+' '+anime_attr['sypnopsis']
combined_features.shape

(16214,)

In [15]:
combined_features

0        Cowboy Bebop Action, Adventure, Comedy, Drama,...
1        Cowboy Bebop: Tengoku no Tobira Action, Drama,...
2        Trigun Action, Sci-Fi, Adventure, Comedy, Dram...
3        Witch Hunter Robin Action, Mystery, Police, Su...
4        Bouken Ou Beet Adventure, Fantasy, Shounen, Su...
                               ...                        
16209    Daomu Biji Zhi Qinling Shen Shu Adventure, Mys...
16210    Mieruko-chan Comedy, Horror, Supernatural Unkn...
16211    Higurashi no Naku Koro ni Sotsu Mystery, Demen...
16212    Yama no Susume: Next Summit Adventure, Slice o...
16213    Scarlet Nexus Action, Fantasy Unknown SCARLET ...
Length: 16214, dtype: object

In [16]:
# converting text data to feature vectors

vectorizer = TfidfVectorizer()

In [17]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [18]:
print(feature_vectors)

  (0, 1547)	0.035066018482358345
  (0, 30526)	0.041722638927078386
  (0, 21673)	0.028467649131014256
  (0, 30166)	0.030365893829818694
  (0, 20609)	0.09779126013819046
  (0, 42131)	0.08929746251886428
  (0, 19393)	0.09017676341276915
  (0, 8047)	0.06603937986945552
  (0, 49689)	0.07406846906802166
  (0, 18549)	0.07213624032731208
  (0, 26211)	0.05511305634990167
  (0, 10709)	0.10985995190984728
  (0, 18943)	0.03973459941434537
  (0, 49656)	0.04727512012510223
  (0, 26393)	0.09092250580101731
  (0, 33943)	0.049860891057986134
  (0, 30295)	0.04058301640636632
  (0, 10155)	0.05438528057094354
  (0, 28306)	0.05673147678000932
  (0, 12371)	0.04294449664803967
  (0, 48107)	0.07406846906802166
  (0, 1434)	0.05284381849182306
  (0, 45782)	0.08402916200250875
  (0, 32570)	0.022950919419115496
  (0, 12874)	0.07345230402815238
  :	:
  (16213, 42577)	0.038812820315895814
  (16213, 49432)	0.054444134069148005
  (16213, 6386)	0.049646711018386
  (16213, 14971)	0.038082910254543266
  (16213, 18422)	0

In [19]:
# Look at the features generated
print(vectorizer.get_feature_names())

['00', '000', '000th', '000マイル', '001', '0011', '0015', '002', '003', '005', '0060', '0068', '007', '0071', '0079', '008', '0080', '0081', '0083', '0087', '0088', '009', '0093', '0095', '0096', '0097', '00f', '01', '0105', '0156', '019', '02', '0209ver', '028', '03', '0375', '038', '04', '0401', '044', '05', '06', '07', '07th', '08', '08th', '08戦争童話', '09', '0ch', '0s', '0verflow', '0あいさつ', '0戦はやと', '10', '100', '1000', '10000', '1000say', '1000年の真宝', '1000年女王', '1001', '100th', '100ばんめのサル', '100万の命の上に俺は立っている', '100万円あったら', '100万年地球の旅', '100億パワーの戦士たち', '100大作戦', '100만년', '101', '1014', '1019', '101st', '101th', '101番目の魔物', '102', '103', '1031', '104', '104th', '106', '107', '108', '108th', '109', '109シネマズマナームービー', '10gauge', '10min', '10min30sec', '10sec', '10th', '10ぴきのかえる', '10万光年の来訪者', '10周年記念完全新作アニメーションpv映像', '10年後の異邦人', '10日で100の出来事', '11', '110', '110th', '111', '112', '1121st', '112th', '113', '114', '1146', '115', '117', '1185', '1189', '119', '1192', '1196', '11eyes', '11pm', 

## 5. Test Our Model and Make Some Recommendation

In [20]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.41443707 0.09491059 ... 0.00494186 0.01212995 0.07645083]
 [0.41443707 1.         0.09058418 ... 0.0141276  0.008195   0.05614721]
 [0.09491059 0.09058418 1.         ... 0.00687864 0.01161339 0.03689127]
 ...
 [0.00494186 0.0141276  0.00687864 ... 1.         0.02200113 0.00766258]
 [0.01212995 0.008195   0.01161339 ... 0.02200113 1.         0.01089984]
 [0.07645083 0.05614721 0.03689127 ... 0.00766258 0.01089984 1.        ]]


In [21]:
print(similarity.shape)

(16214, 16214)


## 6. Input from Users

In [22]:
# Input from user

anime_name = input(' Enter your favourite anime name : ')

 Enter your favourite anime name :  Cowboy


In [23]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = anime_attr['title'].tolist()
print(list_of_all_titles)

['Cowboy Bebop', 'Cowboy Bebop: Tengoku no Tobira', 'Trigun', 'Witch Hunter Robin', 'Bouken Ou Beet', 'Eyeshield 21', 'Hachimitsu to Clover', 'Hungry Heart: Wild Striker', 'Initial D Fourth Stage', 'Monster', 'Naruto', 'One Piece', 'Tennis no Ouji-sama', 'Ring ni Kakero 1', 'School Rumble', 'Sunabouzu', 'Texhnolyze', 'Trinity Blood', 'Yakitate!! Japan', 'Zipang', 'Neon Genesis Evangelion', 'Neon Genesis Evangelion: Death & Rebirth', 'Neon Genesis Evangelion: The End of Evangelion', 'Kenpuu Denki Berserk', 'Koukaku Kidoutai', 'Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen', 'Rurouni Kenshin: Meiji Kenkaku Romantan', 'Rurouni Kenshin: Meiji Kenkaku Romantan - Ishinshishi e no Chinkonka', 'Akira', '.hack//Sign', 'Aa! Megami-sama!', 'Aa! Megami-sama! (TV)', 'Tenshi Kinryouku', 'Kidou Tenshi Angelic Layer', 'Ai Yori Aoshi', 'Appleseed (Movie)', 'Arc the Lad', 'Avenger', 'Beck', 'Blue Gender', 'Chobits', 'Chrno Crusade', 'D.N.Angel', 'D.C.: Da Capo', 'DearS', 'Rozen Maiden', 'Rozen M

In [24]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(anime_name, list_of_all_titles)
print(find_close_match)

['Cowboy Bebop', 'Kowabon', 'Copy']


In [25]:
close_match = find_close_match[0]
print(close_match)

Cowboy Bebop


In [26]:
# finding the index of the anime with title of anime

index_of_the_anime = anime_attr[anime_attr.title == close_match]['index'].values[0]
print(index_of_the_anime)

1


In [27]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_anime]))
print(similarity_score)

[(0, 0.41443707349133024), (1, 0.9999999999999998), (2, 0.09058418491711563), (3, 0.044363913904257694), (4, 0.05658039077307074), (5, 0.06339085848475434), (6, 0.06829706972238735), (7, 0.03778669955650702), (8, 0.06331871642910264), (9, 0.06707841753882986), (10, 0.04813333165209784), (11, 0.0824149957227799), (12, 0.0823828450182513), (13, 0.04746148981179737), (14, 0.06242425876962846), (15, 0.062090134793375475), (16, 0.06594874148835018), (17, 0.07887177587388608), (18, 0.044232196429035565), (19, 0.10739958970560382), (20, 0.0755858411197449), (21, 0.06819561529996507), (22, 0.056913775878798506), (23, 0.0782904049036351), (24, 0.09515939574885225), (25, 0.06964159788317888), (26, 0.052692580264768536), (27, 0.046560057085781724), (28, 0.11278647900733277), (29, 0.07703592897816552), (30, 0.03041082486295337), (31, 0.06139295912085414), (32, 0.06595787900049312), (33, 0.019452916052063986), (34, 0.04718832118185361), (35, 0.07089208995526629), (36, 0.07513222603697299), (37, 0.0

In [28]:
# sorting the movies based on their similarity score

sorted_similar_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_anime)

[(1, 0.9999999999999998), (0, 0.41443707349133024), (3044, 0.34022729104046034), (6703, 0.24212757689270287), (961, 0.13940743847239356), (938, 0.13616498273626787), (9887, 0.13585156570917553), (12387, 0.13356472975700673), (7783, 0.13072942544580426), (12416, 0.1279926580191172), (5000, 0.12722300324315064), (11059, 0.12502841264383258), (2353, 0.12378395241441736), (2646, 0.12282592813746598), (13039, 0.12126563158028693), (2551, 0.12081359723307102), (3440, 0.12033957627409335), (3240, 0.11811758580453399), (158, 0.11792828325927412), (8245, 0.11521928607618044), (14168, 0.11506936769551235), (13412, 0.11500022877634221), (7026, 0.11473211025214416), (2545, 0.11459703098970957), (2123, 0.11332370652131114), (100, 0.11281594772140449), (28, 0.11278647900733277), (498, 0.11273895555888636), (5427, 0.11156094549884908), (5317, 0.1111523757178797), (10581, 0.11031940390084337), (1796, 0.1100906571899666), (2465, 0.10988616227213156), (15961, 0.10960015229353254), (423, 0.10950049859799

In [29]:
# print the name of similar Anime based on the index - Top 10

print('Anime suggested for you : \n')

i = 1

for anime in sorted_similar_anime:
  index = anime[0]
  title_from_index = anime_attr[anime_attr.index==index]['title'].values[0]
  if (i<11):
    print(i, '.',title_from_index)
    i+=1

Anime suggested for you : 

1 . Cowboy Bebop: Tengoku no Tobira
2 . Cowboy Bebop
3 . Cowboy Bebop: Yose Atsume Blues
4 . Cowboy Bebop: Ein no Natsuyasumi
5 . Kenran Butou Sai: The Mars Daybreak
6 . Mutant Turtles: Choujin Densetsu-hen
7 . One Piece Film: Gold
8 . Chibikko Cowboy
9 . Terra Formars
10 . Waga Na wa Cowboy


In [30]:
anime_name = input(' Enter your favourite anime name : ')

list_of_all_titles = anime_attr['title'].tolist()

find_close_match = difflib.get_close_matches(anime_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_anime = anime_attr[anime_attr.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_anime]))

sorted_similar_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Animes suggested for you : \n')

i = 1

for anime in sorted_similar_anime:
  index = anime[0]
  title_from_index = anime_attr[anime_attr.index==index]['title'].values[0]
  if (i<10):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite anime name :  Cowboy


Animes suggested for you : 

1 . Cowboy Bebop: Tengoku no Tobira
2 . Cowboy Bebop
3 . Cowboy Bebop: Yose Atsume Blues
4 . Cowboy Bebop: Ein no Natsuyasumi
5 . Kenran Butou Sai: The Mars Daybreak
6 . Mutant Turtles: Choujin Densetsu-hen
7 . One Piece Film: Gold
8 . Chibikko Cowboy
9 . Terra Formars


# Second Version of Anime Recommendation

In [31]:
# New data
sec_anime_data = ori_data

In [32]:
# Some df info
sec_anime_data.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [33]:
anime_cross_table = pd.crosstab(sec_anime_data["Name"], sec_anime_data["Genres"])
print(anime_cross_table)

Genres                                              Action  Action, Adventure  \
Name                                                                            
"0"                                                      0                  0   
"Aesop" no Ohanashi yori: Ushi to Kaeru, Yokuba...       0                  0   
"Bungaku Shoujo" Kyou no Oyatsu: Hatsukoi                0                  0   
"Bungaku Shoujo" Memoire                                 0                  0   
"Bungaku Shoujo" Movie                                   0                  0   
...                                                    ...                ...   
xxxHOLiC Rou                                             0                  0   
xxxHOLiC Shunmuki                                        0                  0   
Üks Uks                                                  0                  0   
ēlDLIVE                                                  0                  0   
◯                           

## Calculation Jaccard similarity between anime

In [34]:
# import jaccard score

from sklearn.metrics import jaccard_score

In [35]:
sec_anime_data.loc[sec_anime_data ['Name'] == 'One Piece']

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
11,21,One Piece,8.52,"Action, Adventure, Comedy, Super Power, Drama,...",One Piece,ONE PIECE,TV,Unknown,"Oct 20, 1999 to ?",Fall 1999,...,292445.0,166186.0,141755.0,85424.0,35342.0,19019.0,8201.0,3675.0,2616.0,6340.0


In [36]:
# test jaccard score

#one_piece_row = sec_anime_data.loc[sec_anime_data ['Name'] == 'One Piece']
#naruto_row = sec_anime_data.loc[sec_anime_data ['Name'] == 'Naruto']

#Extract rows and convert to numpy array .values method
bus_gamer_row = anime_cross_table.loc['Bus Gamer'].values
haruwo_row = anime_cross_table.loc['Haruwo'].values

print(jaccard_score(bus_gamer_row , haruwo_row))

1.0


In [37]:
#Extract rows and convert to numpy array .values method
one_piece_row = anime_cross_table.loc['One Piece'].values
naruto_row = anime_cross_table.loc['Naruto'].values

print(jaccard_score(one_piece_row , naruto_row))


0.0


Jaccard similarity not viable to use as the result for scored only 1 and 0 .